In [2]:
pip install pydeck

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pydeck as pdk
import re

#버스정류장 위치정보 data
df_coord = pd.read_csv('bus_coord.csv',sep=",", encoding='cp949')
df_coord
lon = df_coord['X좌표']
lat = df_coord['Y좌표']

templist=[]
for i in range(len(lon)):    
    tempcoord=[lon[i],lat[i]]
    templist.append(tempcoord)

df_coord['coordinates'] = templist

#정류소번호 및 노선번호
df_bus_line = pd.read_csv('bus_line.csv',sep=',', encoding='cp949')
df_bus_line.head()
df_bus_line_0 = df_bus_line[['노선번호','버스정류장ARS번호']]
df_bus_line_0.columns=['노선번호','정류소번호']

#합병
df_merge=pd.merge(left=df_bus_line_0, right = df_coord, how ="outer", on = '정류소번호')

#행에서 nans제거
df_merge=df_merge.dropna(subset=['X좌표'])
df_merge=df_merge.dropna(subset=['Y좌표'])
df_merge=df_merge.dropna(subset=['노선번호'])
df_merge=df_merge.dropna(subset=['정류소번호'])
df_merge=df_merge.dropna(subset=['coordinates'])

# 노선 이름 데이터 정리
df_bus_line_1 = df_bus_line_0[['노선번호']]
df_bus_line_2 = df_bus_line_1.drop_duplicates('노선번호')

line_num=[]
for i in range(len(df_bus_line_2)):
    line = df_bus_line_2.iloc[i,0]
    line_num.append(line)

df_geometry = pd.DataFrame(columns=['line_name'])
df_geometry['line_name'] = line_num

df_line = df_geometry.sort_values(by = ['line_name'])

#데이터 묶기
eachgroup = list(df_merge.groupby('노선번호'))
coordinate_dummy=[]
for i in range(len(eachgroup)):
    df_line_nums = eachgroup[i][1]
    df_line_num = df_line_nums.drop_duplicates(['정류소번호'],keep='first')
    series_coords = df_line_num['coordinates']
    arr_coords = series_coords.array
    list_coords = list(arr_coords)
    coordinate_dummy.append(list_coords)
    
def make_color(x): 
    X =re.findall("\d+",x)
    Y= float(X[0])
    Z = re.findall("N", x)
    z=""
    if 0<Y<10 and len(x)==2 : z = [255,255,0]
    elif Z==['N'] : z = [0,0,102]
    elif 0<Y<100 and len(x)>3 : z = [102,255,102]
    elif 100<=Y<1000 : z = [51,102,255]
    elif 1000<Y<9000: z = [51,204,51] 
    elif 9000<Y<10000: z = [255,0,51]  
    return z

def take_weight(x): 
    X =re.findall("\d+",x)
    Y = float(X[0])
    Z = re.findall("N", x)
    z=""
    if 0<Y<10 and len(x)==2 : z = [25]
    elif Z==['N'] : z = [10]
    elif 0<Y<100 and len(x)>3 : z = [25]
    elif 100<=Y<1000: z = [6]
    elif 1000<Y<9000: z = [6] 
    elif 9000<Y<10000: z = [20]  
    return z

df_line['coordinates'] = coordinate_dummy
df_line['color'] = df_line['line_name'].apply(make_color)
df_line['weight'] = df_line['line_name'].apply(take_weight)

#Render

line_layer = pdk.Layer(
    'PathLayer',
    df_line,
    get_path='coordinates',
    get_width ='weight',
    get_color='[color[0],color[1],color[2]]',
)

point_layer = pdk.Layer(
    'ScatterplotLayer',
    df_merge,
    get_position = '[X좌표,Y좌표]',
    get_radius =20,
    get_fill_color = '[255,255,255]',
    pickable = True,
    auto_highlight=True
)


center = [126.9825876,37.5377887]
view_state = pdk.ViewState(longitude=center[0], latitude=center[1], zoom=10)

r = pdk.Deck(layers=[line_layer,point_layer],initial_view_state=view_state)
r.to_html("test_for_bus_stop.html")


'C:\\Users\\User\\Desktop\\jk_coding_study\\test_for_bus_stop.html'